# Cubeviz with WFC3/IR ramp

This concept notebook shows Cubeviz loading HST WFC3/IR ramp FITS file by collating the extensions and casting them as 3D Spectrum1D objects.

In [ ]:
import numpy as np
from astropy import units as u
from astropy.io import fits
from astropy.nddata import StdDevUncertainty
from astropy.utils.data import download_file
from specutils import Spectrum1D

from jdaviz import Cubeviz

In [ ]:
wfc3ir_file = download_file('https://mast.stsci.edu/api/v0.1/Download/file?uri=mast:HST/product/icgk01a8q_ima.fits', cache=True)
fits.info(wfc3ir_file)

In [ ]:
sci_unit = u.electron / u.s  # From BUNIT but its value is invalid for astropy.units
err_unit = sci_unit  # Assume the same

with fits.open(wfc3ir_file) as pf:
    primary_header = pf[0].header
    sci_cube = np.stack([hdu.data for hdu in pf if hdu.name == 'SCI'], axis=2) * sci_unit
    err_cube = np.stack([hdu.data for hdu in pf if hdu.name == 'ERR'], axis=2) * err_unit
    dq_cube = np.stack([hdu.data for hdu in pf if hdu.name == 'DQ'], axis=2) * u.dimensionless_unscaled
    
uncertainty = StdDevUncertainty()
uncertainty.array = err_cube

sci_cube.shape

In [ ]:
sc = Spectrum1D(flux=sci_cube, uncertainty=uncertainty, mask=dq_cube, meta=primary_header)

In [ ]:
cubeviz = Cubeviz()

In [ ]:
cubeviz.load_data(sc, data_label='icgk01a8q_ima')

In [ ]:
cubeviz.show()

For sanity check, we display first slice of `SCI` in Matplotlib. Both displays should look the same.

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
arr = fits.getdata(wfc3ir_file, ext=1)
arr.shape

In [ ]:
plt.imshow(arr, origin='lower', vmin=0, vmax=1000)